# Automate the degree of consolidation using Python


In [1]:
#load in the path to the Plaxis 
import imp
plaxis_path = r'C:\Program Files (x86)\Plaxis\PLAXIS 2D 2016'
found_module = imp.find_module('plxscripting', [plaxis_path])
plxscripting = imp.load_module('plxscripting', *found_module)
from plxscripting.easy import *
from scipy.interpolate import griddata
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
port = 10001
s_o, g_o = new_server('localhost', port)
s_o.open(r'C:\Users\jeffrey.kwong\Desktop\Python\SC model\MCC_AxSC_UPMW_r02.p2dx')

'OK'

In [3]:
def get_avg_epwp(Phase,m,n,xbound,ybound):
    '''
    This function returns the average excess pore water pressure at a certain phase
    Parameter:
    phase_df: Phase of interested, example 'g_o.Phase_6'
    m: number of discretisation in the x-direction
    n: number of discretisation in the y-direction
    xbound: x boundary of the soil wanted to be considered
    ybound: y boundary of the soil wanted to be considered
    '''

    x = np.array(g_o.getresults(Phase, g_o.ResultTypes.Soil.X, "node"))
    y = np.array(g_o.getresults(Phase, g_o.ResultTypes.Soil.Y, "node"))
    epwp =np.array(g_o.getresults(Phase, g_o.ResultTypes.Soil.PExcess, "node"))

    ux = np.array(g_o.getresults(Phase, g_o.ResultTypes.Soil.Ux, "node"))
    uy = np.array(g_o.getresults(Phase, g_o.ResultTypes.Soil.Ux, "node"))

    df = pd.DataFrame(dict(x=[],y=[],epwp=[]))
    df.x = x-ux; df.y = y-uy; df.epwp = epwp
    
    
    
    dfMDx=[]
    dfMDy=[]
    dfMDepwp=[]

    for i in range (len(df.x)):
            if  df.x[i] >= xbound and df.y[i] <= ybound:
                dfMDx.append(df.x[i])
                dfMDy.append(df.y[i])
                dfMDepwp.append(df.epwp[i])
            
    df_MD = pd.DataFrame(dict(x=[],y=[],epwp=[]))
    df_MD.x = dfMDx
    df_MD.y = dfMDy
    df_MD.epwp = dfMDepwp


    
    R = max(df_MD.x)-min(df_MD.x)
    change_r = R/n
    Z = max(df_MD.y)-min(df_MD.y)
    change_z = Z/m
    
    area = change_r * change_z
    
    newx = np.arange(min(df_MD.x)+change_r/2,max(df_MD.x)+change_r+change_r/2,change_r)
    newy = np.arange(min(df_MD.y)+change_z/2,max(df_MD.y)+change_z+change_z/2,change_z)
    grid_x, grid_y = np.meshgrid(newx,newy)
    coord = (df_MD.x,df_MD.y)
#import pdb; pdb.set_trace()
    zi = griddata(coord,df_MD.epwp,(grid_x.flatten(),grid_y.flatten()),method='linear')
    
    zi_r = np.array(grid_x.flatten())*np.array(zi)
    
    n_nans = len(np.where(np.isnan(zi)))
    H=max(df_MD.y)-min(df_MD.y)
    total_area = np.absolute((max(df_MD.x)*max(df_MD.x)-min(df_MD.x)*min(df_MD.x))*H)
    zi_real = zi_r[np.invert(np.isnan(zi))]
    avg_epwp = 2*area*sum(zi_real)/total_area
    
    return avg_epwp

In [5]:
avg_epwp = get_avg_epwp(g_o.Phase_6,20,20,0.5,-11)
print(avg_epwp)

-62.987160143223484


# Calculate the Avarege Excess Pore Water Pressure 


$$u_{avg} = \frac{1}{V} \iiint_V u(r,z)r\cdot d\theta dzdr = \frac{1}{V} \iint_V u(r,z) 2\pi rdrdz =  \frac{2\pi}{V} \iint_V u(r,z) rdrdz$$

The discretization of the integration gives:

$$ u_{avg} = \frac{2}{ (R^2 - r_o^2)H} \sum_i^m\sum_j^n (u_{ij} r_j \Delta r_{ij} \Delta z_{ij})$$

where n - number of discretization, R - radius of soil column and $r_o$ - radius of well

As we are using regular mesh, $\Delta z_{ij}$ and $\Delta r_{ij}$ are constant, the discrete form of integration could be further simplified as:

$$ u_{avg} = \frac{2\Delta r \Delta z}{ (R^2 - r_o^2)H} \sum_i^m\sum_j^n u_{ij} r_j  = C \sum_i^m\sum_j^n u_{ij} r_j$$

where $C$ is a constant once discretization is estabilished and $ C = \frac{2\Delta r \Delta z}{ (R^2 - r_o^2)H}$

In deriving the averge degree of consolidation, we will overlay a structured $m\times n$ mesh on top of the the existing output from Plaxis. The nodal values will be calculated using interpolation. It's expected that as n gets larger, the calculated average u will converge to a constant.